In [1]:
# %load guitar.py
import random
import math
import time
import Quantum_randomNum
SAMPLE_RATE = 44100
DECAY_FACTOR = 0.996


class GuitarString:
    def __init__(self, frequency):
        if frequency <= 0:
            raise ValueError("Frequency must be positive.")
        self.frequency = frequency
        self.tic_time = 0
        self.queue = [0] * int(round(SAMPLE_RATE / frequency))

    def pluck(self):
        for i in range(len(self.queue)):
            self.queue[i] = generate_random_number(-0.5, 0.5)

    def tic(self):
        first = self.queue[0]
        self.queue = self.queue[1:]
        second = self.queue[0]
        new_term = DECAY_FACTOR * 0.5 * (first + second)
        self.queue.append(new_term)
        self.tic_time += 1

    def sample(self):
        return self.queue[0]

    def get_time(self):
        return self.tic_time

    def get_frequency(self):
        return self.frequency

In [2]:
# %load GuitarMusic.py

SAMPLE_RATE = 44100
import sys
import math
from guitar import GuitarString


CONCERT_A = 440.0
NUM_STRINGS = 37
STEP = 1 / SAMPLE_RATE


def open_files():
    # open input data file
    in_file_name = input("Enter the name of the input file: ")
    try:
        infile = open(in_file_name, "r")
    except FileNotFoundError:
        print("Error opening input data file")
        sys.exit(1)

    # open output data file
    out_file_name = input("Enter the name of the output file: ")
    try:
        outfile = open(out_file_name, "w")
    except FileNotFoundError:
        print("Error opening output data file")
        sys.exit(1)

    return infile, outfile


def close_files(infile, outfile):
    infile.close()
    outfile.close()


def create_strings():
    strings = []
    for i in range(NUM_STRINGS):
        factor = pow(2, (i - 24) / 12.0)
        frequency = CONCERT_A * factor
        string = GuitarString(frequency)
        strings.append(string)
    return strings


def sum_samples(strings):
    sample_sum = 0
    for string in strings:
        sample_sum += string.sample()
    return sample_sum


def process_file(strings, infile, outfile):
    time_count = 0
    for line in infile:
        read_time, gtr_key = line.strip().split()
        read_time = float(read_time)
        gtr_key = int(gtr_key)

        while time_count < read_time:
            outfile.write("  " + str(time_count) + "\t" + str(sum_samples(strings)) + "\n")
            for string in strings:
                string.tic()
            time_count += STEP

        if gtr_key == -1:
            return

        strings[gtr_key].pluck()
        print(".", end="")
        sys.stdout.flush()


def main():
    strings = create_strings()
    infile, outfile = open_files()

    outfile.write("; Sample Rate " + str(SAMPLE_RATE) + "\n")
    outfile.write("; Channels 1\n")

    print("Reading the input file and generating a .dat file for sox")

    process_file(strings, infile, outfile)

    close_files(infile, outfile)

    print("Done.")


if __name__ == "__main__":
    main()

Enter the name of the input file: song.txt
Enter the name of the output file: first.txt
Reading the input file and generating a .dat file for sox
....Done.


In [3]:
import qiskit

In [53]:
# %load Quantum_randomNum.py
from qiskit import QuantumCircuit, execute, Aer

def generate_random_number(start, end):
    
    random = 0.0
    n = 5
    
    

    for p in range(n):

            # Determine the number of qubits required to represent the range
        num_qubits = len(bin(end - 2)[2:])

            # Create a quantum circuit with the necessary number of qubits
        qc = QuantumCircuit(num_qubits, num_qubits)

            # Apply Hadamard gates to create a superposition of all possible values
        for i in range(num_qubits):
            qc.h(i)

            # Apply a series of controlled-X gates to set the range
        for i in range(num_qubits - 2):
            qc.ccx(i, i + 1, num_qubits - 1)

            # Measure the qubits
        qc.measure(range(num_qubits), range(num_qubits))

            # Set up the backend for simulation
        backend = Aer.get_backend('qasm_simulator')

            # Execute the circuit and get the results
        job = execute(qc, backend, shots=1)
        result = job.result()
        counts = result.get_counts(qc)

            # Extract the random number and map it to the desired range
        random_number = int(list(counts.keys())[0], 2)
        random_number = start + random_number % (end - start) 
        
        
        random =  random + random_number * 10 ** (-p)
    
    
    return "{:.{}f}".format(random, 4)
# Example usage

# start = 5
# end = -5
# random_number = generate_random_number(start, end)
# print("Random number between", start, "and", end, ":", random_number)



Random number between 5 and -5 : 2.6357
